In [17]:
# importing the threading module
from datetime import date
import pandas as pd
import time
import sys
# local modules
from binance.client import Client
from binance.enums import *
from indicator import indicators
# local file
import secrets
import json
import yfinance as yf
import numpy as np
import datetime as dt
import requests 
import json 
import pandas as pd 
import numpy as np  
import requests
import time
import urllib
from finta import TA
from multiprocessing import Process
import threading
from NeatLogger import Log

In [3]:
df3 = pd.read_csv("parameters.csv")
df3.set_index("parameters",inplace=True)
df1=df3.T

In [35]:
# To add a new cell, type '# %%'
# To add a new markdown cell, type '# %% [markdown]'

# %%


from datetime import date
import pandas as pd
import time
import sys
# local modules
from binance.client import Client
from binance.client import BaseClient
from binance.enums import *
from indicator import indicators
# local file
import secrets
import json
import yfinance as yf
import numpy as np
import datetime as dt
import requests 
import json 
import pandas as pd 
import numpy as np  
import requests
import time
import urllib
from finta import TA

# %%

df3 = pd.read_csv("parameters.csv")
df3.set_index("parameters",inplace=True)
df2=df3.T

import logging
import sys

logger = logging.getLogger()
logger.setLevel(logging.INFO)
formatter = logging.Formatter('%(asctime)s | %(levelname)s | %(message)s')

stdout_handler = logging.StreamHandler(sys.stdout)
stdout_handler.setLevel(logging.DEBUG)
stdout_handler.setFormatter(formatter)

file_handler = logging.FileHandler('logs.log')
file_handler.setLevel(logging.DEBUG)
file_handler.setFormatter(formatter)


logger.addHandler(file_handler)
logger.addHandler(stdout_handler)


# import telepot
# bot = telepot.Bot('1715056219:AAGxytb3U1gIt1vlVn8Jf5b4za3E1HPuOd4')
# bot.getMe()

# %%
# client = Client((df2['binance_api_key'][0]), str(df2['binance_api_secret_key'][0]))
BaseClient.FUTURES_TESTNET_URL='https://fapi.binance.{}/fapi'
BaseClient.FUTURES_URL = 'https://testnet.binancefuture.com/fapi'


client = Client("9358c5e083cedf0f310b4dd17c1c2be8760b2752a8ca890046c994da91a37b9c", "fa8e91b3077b01fc9f7fba888495b1a9f919a033e13cc51221fdb8e275c25496")


# %%
# while True:
#     data=client.get_all_tickers()
#     symbol=[]
#     prices=[]

#     df=pd.DataFrame()
#     for dat in data:
#         symbol.append(dat['symbol'])
#         prices.append(dat['price'])
        

#     df['symbols']=np.array(symbol)
#     df['price']=np.array(prices)
#     df.to_csv('data.csv')


# %%
# import json

# while True:
#     try:
#         data=client.get_all_tickers()
#         # Data to be written
#         dictionary =data

#         # Serializing json 
#         json_object = json.dumps(dictionary, indent = 2)
        
#         # Writing to sample.json
#         with open("sample.json", "w") as outfile:
#             outfile.write(json_object)
#     except Exception as e:
#         print(e)


# %%

# %%




def StochRSI(series, period=14, smoothK=3, smoothD=3):
    # Calculate RSI 
    delta = series.diff().dropna()
    ups = delta * 0
    downs = ups.copy()
    ups[delta > 0] = delta[delta > 0]
    downs[delta < 0] = -delta[delta < 0]
    ups[ups.index[period-1]] = np.mean( ups[:period] ) #first value is sum of avg gains
    ups = ups.drop(ups.index[:(period-1)])
    downs[downs.index[period-1]] = np.mean( downs[:period] ) #first value is sum of avg losses
    downs = downs.drop(downs.index[:(period-1)])
    rs = ups.ewm(com=period-1,min_periods=0,adjust=False,ignore_na=False).mean() /          downs.ewm(com=period-1,min_periods=0,adjust=False,ignore_na=False).mean() 
    rsi = 100 - 100 / (1 + rs)

    # Calculate StochRSI 
    stochrsi  = (rsi - rsi.rolling(period).min()) / (rsi.rolling(period).max() - rsi.rolling(period).min())
    stochrsi_K = stochrsi.rolling(smoothK).mean()
    stochrsi_D = stochrsi_K.rolling(smoothD).mean()

    return stochrsi, stochrsi_K, stochrsi_D

# %%
def candle(symbol, interval):

    global df2
    root_url = 'https://api.binance.com/api/v1/klines'
    url = root_url + '?symbol=' + symbol + '&interval=' + interval
    data = json.loads(requests.get(url).text)
    df = pd.DataFrame(data)
    df.columns = ['Datetime',
                'Open', 'High', 'Low', 'Close', 'volume',
                'close_time', 'qav', 'num_trades',
                'taker_base_vol', 'taker_quote_vol', 'ignore']
    df.index = [dt.datetime.fromtimestamp(x / 1000.0) for x in df.close_time]
    
    df.drop(['close_time','qav','num_trades','taker_base_vol', 'taker_quote_vol', 'ignore'],axis=1,inplace=True)
           
    
    
    df['Open']=pd.to_numeric(df["Open"], downcast="float")
    df["High"]=pd.to_numeric(df["High"], downcast="float")
    df["Low"]=pd.to_numeric(df["Low"], downcast="float")
    df["Close"]=pd.to_numeric(df["Close"], downcast="float")
    df["volume"]=pd.to_numeric(df["volume"], downcast="float")
    df['ATR']=TA.ATR(df,int(df2['ATR'][0]))
    df1=pd.DataFrame()
    Open=[]
    
    df1['Close']=((df['Open'] + df['High'] + df['Low'] + df['Close'])/4)
    for i in range(len(df)):
        if i==0:
            Open.append(0)

        else:
            Open.append((df['Open'][i-1]+df['Close'][i-1])/2)
    df1['Open']=np.array(Open)
    df1['volume']=df['volume']
    df1['ATR']=df['ATR']
    High=[]
    Low=[]
    High.append(0)
    Low.append(0)
    for i in range(1,len(df)):

        High.append(max(df['High'][i],df1['Close'][i],df1['Open'][i]))
        Low.append(min(df['Low'][i],df1['Close'][i],df1['Open'][i]))

    df1['High']=np.array(High)
    df1['Low']=np.array(Low)

    df1['Stoch-k']=StochRSI(df1['Close'],int(df2['STOCH-PERIOD'][0]),int(df2['STOCH-K'][0]),int(df2['STOCH-D'][0]))[1]*100
    df1['Stoch-d']=StochRSI(df1['Close'],int(df2['STOCH-PERIOD'][0]),int(df2['STOCH-K'][0]),int(df2['STOCH-D'][0]))[2]*100
    df1['RSI']=TA.RSI(df1,int(df2['RSI'][0]))

    # df1['STOCH-k']=TA.STOCH(df1,14)
    # df1['STOCH-D']=TA.STOCHD(df1,14)

    return df1[:-1]


In [42]:
df=candle('ETHUSDT','5m')



In [55]:
str(df.index[-2])

'2022-01-18 11:24:59.999000'

In [45]:
import logging

In [32]:
import logging
import sys

logger = logging.getLogger()
logger.setLevel(logging.INFO)
formatter = logging.Formatter('%(asctime)s | %(levelname)s | %(message)s')

stdout_handler = logging.StreamHandler(sys.stdout)
stdout_handler.setLevel(logging.DEBUG)
stdout_handler.setFormatter(formatter)

file_handler = logging.FileHandler('logs.log')
file_handler.setLevel(logging.DEBUG)
file_handler.setFormatter(formatter)


logger.addHandler(file_handler)
logger.addHandler(stdout_handler)

# self.logger = logging.getLogger('urbanGUI')

In [33]:
logger.info('This is a log message!')

INFO:root:This is a log message!
[timestamp 2022-01-17 23:02:57.177621+05:30] [name root] [levelname INFO] [filename <ipython-input-33-4799c163a70e>] [lineno 1] This is a log message!
[timestamp 2022-01-17 23:02:57.177621+05:30] [name root] [levelname INFO] [filename <ipython-input-33-4799c163a70e>] [lineno 1] This is a log message!
[timestamp 2022-01-17 23:02:57.177621+05:30] [name root] [levelname INFO] [filename <ipython-input-33-4799c163a70e>] [lineno 1] This is a log message!
[timestamp 2022-01-17 23:02:57.177621+05:30] [name root] [levelname INFO] [filename <ipython-input-33-4799c163a70e>] [lineno 1] This is a log message!


2022-01-17 23:02:57,177 | INFO | This is a log message!


In [3]:
from multiprocessing import Process
import time


def f(name):
    time.sleep(2)
    print('hi')


Processes=[]

for i in range(5):
    Processes.append(Process(target=f, args=('bob',)))



for process in Processes:
    process.start()

for process in Processes:

    process.join()

registering process 0


In [11]:
from multiprocessing import Process
import os

def info(title):
    print(title)
    print('module name:', __name__)
    print('parent process:', os.getppid())
    print('process id:', os.getpid())

def f(name):
    info('function f')
    print('hello', name)

if __name__ == '__main__':
    # info('main line')
    p = Process(target=f, args=('bob',))
    p.start()
    p.join()

In [1]:
import multiprocessing as mp
import math
import os

#----------------------------------------------------------------------------------------------------------------------
#This is a VERY SIMPLE example of parallel processing in Python using the multiprocessing library using the Process object.
# Details of implementation/tutorial can be found here: http://madhugnadig.com/articles/parallel-processing/2017/01/25/parallel-programming-in-python-with-ease.html
#----------------------------------------------------------------------------------------------------------------------


def process_info():
    print('Module:' + str(__name__) + '\n')
    print('Parent Process id:' + str(os.getppid())+ '\n' )
    print('Process id:' + str(os.getpid())+ '\n\n' )

def cubes_and_sqare_root(a,x,output):
	process_info()
	output.put((int(x), math.sqrt(a**3)))

def main():
	#Using the queue as the message passing paradigm 
	output = mp.Queue()
	processes = [mp.Process(target=cubes_and_sqare_root, args=(x, x,output)) for x in range(1,8)]

	for process in processes:
		process.start()

	for process in processes:
		process.join()

	results = [output.get() for process in processes]
	
	print(results)

if __name__ == '__main__':    
	main()

In [1]:
import re
tweet='$AMD #TWTradeAlert\n\nBTO 21JAN22 125P @ $3.95\n\n*starter swing*'
data=re.split(' |\n',tweet)

In [3]:
print(data)

['$AMD', '#TWTradeAlert', '', 'BTO', '21JAN22', '125P', '@', '$3.95', '', '*starter', 'swing*']


In [8]:
price=1.2
formatted_float = "{:.2f}".format(price)

In [9]:
formatted_float

'1.20'